# zone_v7

In [1]:
import os, sys, time, json, requests, re, random, string, argparse, logging, copy, glob, pathlib, shutil
import hashlib
import numpy as np
import pandas as pd
# import japanize_matplotlib
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg
%matplotlib inline
# import xml.etre.ElementTree as ET
from xml.dom import minidom
from lxml import etree

font_path = '/System/Library/Fonts/ヒラギノ角ゴシック W4.ttc'
font_prop = fm.FontProperties(fname=font_path)
plt.rcParams['font.family'] = font_prop.get_name()

In [2]:
bucket_url = "https://r2-image-uploader.danjiri.workers.dev/"

def generate_root():
    root = etree.Element("card-stack", 
                    attrib={
                        "location.name": "table",
                        "location.x": "825",
                        "location.y": "1200",
                        "poxZ": "0",
                        "rotate": "0",
                        "zindex": "167",
                        "owner": "",
                        "isShowTotal": "true"
                        })
    card_stack = etree.SubElement(root, "data", attrib={"name": "card-stack"})
    image = etree.SubElement(card_stack, "data", attrib={"name": "image"})
    etree.SubElement(image, "data", attrib={"type": "image", "name": "imageIdentifier"}).text = ""
    common = etree.SubElement(card_stack, "data", attrib={"name": "common"})
    etree.SubElement(common, "data", attrib={"name": "name"}).text = "cards_stack"
    etree.SubElement(card_stack, "data", attrib={"name": "detail"}).text = ""
    return root


def create_card(card_root, face, back, name=""):
    card = etree.SubElement(card_root, "card", attrib={
        "location.name": "table",
        "location.x": "875",
        "location.y": "1525",
        "poxZ": "0",
        "state": "0",
        "rotate": "0",
        "owner": "",
        "zindex": "0"
        })
    card_data = etree.SubElement(card, "data", attrib={"name": "card"})
    image_data = etree.SubElement(card_data, "data", attrib={"name": "image"})
    etree.SubElement(image_data, "data", attrib={"type": "image", "name": "imageIdentifier"}).text = name
    etree.SubElement(image_data, "data", attrib={"type": "image", "name": "front"}).text = bucket_url + face
    etree.SubElement(image_data, "data", attrib={"type": "image", "name": "back"}).text = bucket_url + back
    common_data = etree.SubElement(card_data, "data", attrib={"name": "common"})
    etree.SubElement(common_data, "data", attrib={"name": "name"}).text = "カード"
    etree.SubElement(common_data, "data", attrib={"name": "size"}).text = "2"
    etree.SubElement(card_data, "data", attrib={"name": "detail"}).text = ""


def create_xml_file(df, name="test", output_dir="output/xml"):
    root = generate_root()
    card_root = etree.SubElement(root, "node", attrib={"name": "cardRoot"})
    for i, row in df.iterrows():
        face = row["face"]
        back = row["back"]
        create_card(card_root, face, back)

    # 文字列に変換して整形
    xml_str = etree.tostring(root, encoding="utf-8", pretty_print=True, xml_declaration=True)

    # ファイル保存
    with open(os.path.join(output_dir, f"{name}.xml"), "wb") as f:
        f.write(xml_str)

## 画像作成

In [3]:
file = "input/testplay.xlsx"
output_dir = "output/img"
materials_dir = "materials"

open_df = pd.read_excel(file, sheet_name="open")
open_list = []
for reverse, group in open_df.groupby("reverse"):
    type_effect_dict = dict(zip(group["type"], group["effect"]))
    open_list.append(type_effect_dict)

cards_df = pd.read_excel(file, sheet_name="cardpool")

icon_name_list = ["金", "木", "海"]
icon_img_list = ["money", "wood", "sea", "question"]
icon_img_list = [mpimg.imread(os.path.join(materials_dir, f"{i}.png")) for i in icon_img_list]
normal_icon_img_list = icon_img_list[:-1]
any_icon_img = icon_img_list[-1]
counter = 0
fs = 9
height = 3.50; width = 2.48
hv_ratio = width / height
box_width = 0.15
box_height = box_width * hv_ratio
box_margin = 0.02
engine_box_width = 0.2
engine_box_height = engine_box_width * hv_ratio

cards_df.head()

,exclude,level,engine,e1,e2,e3,e4,r1,r2,cost,...,c4,c5,c6,c7,c8,otype,score,condition,effect,value
0,NaN,1,1,0,NaN,NaN,NaN,0,NaN,2,...,NaN,NaN,NaN,NaN,NaN,n,1,NaN,NaN,NaN
1,NaN,1,1,0,NaN,NaN,NaN,0,NaN,2,...,NaN,NaN,NaN,NaN,NaN,n,1,NaN,NaN,NaN
2,NaN,1,1,0,NaN,NaN,NaN,0,NaN,2,...,NaN,NaN,NaN,NaN,NaN,n,1,NaN,NaN,NaN
3,NaN,1,1,0,NaN,NaN,NaN,0,NaN,2,...,NaN,NaN,NaN,NaN,NaN,c,1,NaN,NaN,NaN
4,NaN,1,1,0,NaN,NaN,NaN,0,NaN,2,...,NaN,NaN,NaN,NaN,NaN,c,1,NaN,NaN,NaN


In [4]:
# 道カードの裏面
reverse_box_width = 0.5
reverse_box_height = reverse_box_width * hv_ratio
reverse_box_margin = 0.1

for level in range(1, 3):
    for i, img in enumerate(normal_icon_img_list):
        fig, ax = plt.subplots(figsize=(width, height))
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)

        for j in range(level):
            y_start = 0.5 - level*reverse_box_height/2 - (level-1)*reverse_box_margin/2 + j*(reverse_box_height+reverse_box_margin)
            ax.imshow(img, extent=(0.5 - reverse_box_width/2, 0.5 + reverse_box_width/2, y_start, y_start + reverse_box_height), aspect="auto")

        ax.axis("off")
        fig.savefig(os.path.join(output_dir, f"back{level}{i}.png"), dpi=300)
        plt.close(fig)

In [5]:
# その他の裏面
name_list = ["action", "objective", "research", "ability", "score"]
for name in name_list:
    fig, ax = plt.subplots(figsize=(width, height))
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)

    ax.text(0.5, 0.5, name, ha="center", va="center", fontsize=16)

    ax.axis("off")
    fig.savefig(os.path.join(output_dir, f"{name}_rev.png"), dpi=300)
    plt.close(fig)

In [6]:
# cards
fr_list = []

def get_img(j, diff):
    if diff == -1:
        return any_icon_img
    else:
        return normal_icon_img_list[int(j+diff)%3]

for i, row in cards_df.iterrows():
    level = row["level"]
    for j in range(3):
        base_icon_name = icon_name_list[j]
        fig, ax = plt.subplots(figsize=(width, height))
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)

        ax.text(0, 1, "COST", horizontalalignment='left', verticalalignment='bottom', fontsize=fs)
        cost_list = [row[f"c{k}"] for k in range(1,7)]
        for k, cost_diff in enumerate(cost_list):
            if np.isnan(cost_diff):
                break
            img = get_img(j, cost_diff)
            kx, ky = k//3, k%3
            ax.imshow(img, extent=(0.02 + kx*box_width, 0.02 + (kx+1)*box_width, 0.98 - (ky+1)*box_height, 0.98 - ky*box_height), aspect="auto")

        ax.text(1, 1, "ENGINE", horizontalalignment='right', verticalalignment='bottom', fontsize=fs)
        engine_list = [row[f"e{k}"] for k in range(1,4)]
        for k, engine_diff in enumerate(engine_list):
            if np.isnan(engine_diff):
                break
            img = get_img(j, engine_diff)
            kx, ky = k%2, k//2
            ax.imshow(img, extent=(0.98 - (kx+1)*engine_box_width, 0.98 - kx*engine_box_width, 0.98 - (ky+1)*engine_box_height, 0.98 - ky*engine_box_height), aspect="auto")


        ax.text(0, 0.5, "EFFECT", horizontalalignment='left', verticalalignment='bottom', fontsize=fs)
        condition = row["condition"]
        effect = row["effect"]
        if not pd.isna(effect):
            text = effect if pd.isna(condition) else f"{condition}\n{effect}"
            text = text.replace("〜", base_icon_name)
            ax.text(0, 0.46, text, horizontalalignment='left', verticalalignment='top', fontsize=fs)

        r1 = row["r1"]
        reverse_id = int(j+r1)%3
        reverse_img = normal_icon_img_list[reverse_id]

        ax.text(0, 0.2, "OPEN", horizontalalignment='left', verticalalignment='bottom', fontsize=fs)
        opentype = row["otype"]
        text = open_list[reverse_id][opentype]
        if not pd.isna(text):
            ax.text(0, 0.16, text, horizontalalignment='left', verticalalignment='top', fontsize=fs)

        ax.text(1, 0.62, "RECYCLE", ha="right", va="bottom", fontsize=fs)
        for k in range(level):
            ax.imshow(reverse_img, extent=(0.98 - (k+1)*box_width, 0.98 - k*box_width, 0.5, 0.5 + box_height), aspect="auto")

        ax.text(1, 0.2, "SCORE", ha="right", va="bottom", fontsize=fs)
        score = row["score"]
        ax.text(1, 0.18, score, ha="right", va="top", fontsize=fs)

        fr_elem = {
            "type": f"card{level}",
            "face": f"card{counter}.png",
            "back": f"back{level}{reverse_id}.png"
        }
        fr_list.append(fr_elem)

        ax.axis("off")
        fig.savefig(os.path.join(output_dir, f"card{counter}.png"), dpi=300)
        counter += 1

        plt.close(fig)
fr_df = pd.DataFrame(fr_list)
for level in range(1,3):
    current_df = fr_df[fr_df["type"] == f"card{level}"]
    create_xml_file(current_df, f"card{level}")

In [7]:
# 道アクションカード
action_df = pd.read_excel(file, sheet_name="action")
fr_list = []
for i, row in action_df.iterrows():
    fig, ax = plt.subplots(figsize=(width, height))
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    
    level = row["level"]
    effect = row["effect"]
    ax.text(0, 1, effect, ha="left", va="top", fontsize=13)

    ax.axis("off")
    fig.savefig(os.path.join(output_dir, f"action{i}.png"), dpi=300)
    plt.close(fig)

    fr_elem = {
        "type": "action",
        "face": f"action{i}.png",
        "back": "action_rev.png"
    }
    fr_list.append(fr_elem)
action_fr_df = pd.DataFrame(fr_list)
create_xml_file(action_fr_df, "action")

In [8]:
# 調査カード
research_df = pd.read_excel(file, sheet_name="research")
fr_list = []
for i, row in research_df.iterrows():
    fig, ax = plt.subplots(figsize=(width, height))
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    
    level = row["level"]
    cost = row["cost"]
    ax.text(0, 1, f"Level{level}", ha="left", va="top", fontsize=16)
    ax.text(0.5, 0.5, cost, ha="center", va="center", fontsize=16)

    ax.axis("off")
    fig.savefig(os.path.join(output_dir, f"research{i}.png"), dpi=300)
    plt.close(fig)

    fr_elem = {
        "type": "research",
        "face": f"research{i}.png",
        "back": "research_rev.png"
    }
    fr_list.append(fr_elem)
research_fr_df = pd.DataFrame(fr_list)
create_xml_file(research_fr_df, "research")
# research_fr_df.head()

In [9]:
# 能力カード
ability_df = pd.read_excel(file, sheet_name="ability")
fr_list = []
for i, row in ability_df.iterrows():
    fig, ax = plt.subplots(figsize=(width, height))
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    
    level = row["level"]
    condition = row["condition"]
    effect = row["effect"]
    ax.text(0, 1, f"Level{level}", ha="left", va="top", fontsize=16)
    ax.text(0.5, 0.5, f"{condition}\n{effect}", ha="center", va="center", fontsize=10)

    ax.axis("off")
    fig.savefig(os.path.join(output_dir, f"ability{i}.png"), dpi=300)
    plt.close(fig)

    fr_elem = {
        "type": "ability",
        "face": f"ability{i}.png",
        "back": "ability_rev.png"
    }
    fr_list.append(fr_elem)
fr_df = pd.DataFrame(fr_list)
create_xml_file(fr_df, "ability")
# fr_df.head()

In [10]:
# 目標カード
objective_df = pd.read_excel(file, sheet_name="objective")
fr_list = []
for i, row in objective_df.iterrows():
    fig, ax = plt.subplots(figsize=(width, height))
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    
    condition = row["condition"]
    ax.text(0.5, 0.5, condition, ha="center", va="center", fontsize=12)

    ax.axis("off")
    fig.savefig(os.path.join(output_dir, f"objective{i}.png"), dpi=300)
    plt.close(fig)

    fr_elem = {
        "type": "objective",
        "face": f"objective{i}.png",
        "back": "objective_rev.png"
    }
    fr_list.append(fr_elem)
objective_fr_df = pd.DataFrame(fr_list)
create_xml_file(objective_fr_df, "objective")
# objective_fr_df.head()

In [11]:
# スコアカード
score_df = pd.read_excel(file, sheet_name="score")
fr_list = []
for i, row in score_df.iterrows():
    fig, ax = plt.subplots(figsize=(width, height))
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    
    score = row["score"]
    ax.text(0.5, 0.5, score, ha="center", va="center", fontsize=12)

    ax.axis("off")
    fig.savefig(os.path.join(output_dir, f"score{i}.png"), dpi=300)
    plt.close(fig)

    fr_elem = {
        "type": "score",
        "face": f"score{i}.png",
        "back": "score_rev.png"
    }
    fr_list.append(fr_elem)
fr_df = pd.DataFrame(fr_list)
create_xml_file(fr_df, "score")
# fr_df.head()